# TP 2-3 : Branch-and-bound applied to a knapsack problem

### Initialisation (à faire une seule fois)

In [1]:
import Pkg; 
Pkg.add("GraphRecipes"); Pkg.add("Plots"); 
using GraphRecipes, Plots #only used to visualize the search tree at the end of the branch-and-bound

    Updating registry at `~/.julia/registries/General.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


### Récupération des données

In [2]:
function readKnaptxtInstance(filename)
    price=[]
    weight=[]
    KnapCap=[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end 
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

readKnaptxtInstance (generic function with 1 method)

### Tests de sondabilités TA, TO et TR basés sur le modèle linéaire

In [19]:
function TestsSondabilite_LP(model2, BestProfit, Bestsol)
    TA, TO, TR = false, false, false
    if(model2.infeasible)#Test d'admissibilité : réussi si la capacité restante est négative
        TA=true
        println("TA")
        
    elseif(model2.bound < BestProfit) #Test d'optimalite
        TO=true
        println("TO")
    elseif(verifTR(model2)) #Test de resolution
        TR=true
        println("TR")
        #if (value(benef) >= BestProfit)
        actualProfit = actualPrice(model2)
        if (actualProfit >= BestProfit)
            Bestsol = 1000
            #BestProfit=value(benef)
            BestProfit= actualProfit
            println("\nNew Solution memorized ", Bestsol, " with bestprofit ", BestProfit, "\n")
        end
    else
        println("non sondable")
    end
    TA, TO, TR, Bestsol, BestProfit
end

TestsSondabilite_LP (generic function with 1 method)

## Procédure de séparation (branching) et stratégie d'exploration permettant de se placer au prochain noeud à traiter

In [23]:

function SeparerNoeud_lexicographic_depthfirst!(model2, listobjs, listvals, n)
    # this node is non-sondable. Apply the branching criterion to separate it into two subnodes
    # and choose the child-node at the left  
    
    # lexicographic branching criterion: branch on the 1st object not yet fixed
    bestObj = model2.variables[1] # Objet ayant le meilleur ratio
    for obj in model2.variables
        if obj.ratio > bestObj.ratio && !obj.lock
            bestObj = obj
            
        end
    end
    lockObj(model2, bestObj, 1.0)

    println("\nbranch on object ", bestObj, "\n")

    # depthfirst exploration strategy: the node selected will be the most left of the child-nodes just created
    push!(listobjs,bestObj) #save the identity of the object selected for branching
    push!(listvals,1.0) #save the node selected, identified by the value assigned to the variable/object chosen
    return listobjs, listvals
end


function ExplorerAutreNoeud_depthfirst!(model2, listobjs, listvals, listnodes)
    #this node is sondable, go back to parent node then right child if possible
    
    stop=false
    #check if we are not at the root node
    if (length(listobjs)>= 1)
        #go back to parent node
        obj=pop!(listobjs)
        theval=pop!(listvals)
        tmp=pop!(listnodes)
        freeObj(model2, obj)

        #go to right child if possible, otherwise go back to parent
        while( (theval==0.0) && (length(listobjs)>= 1))
            obj=pop!(listobjs)
            theval=pop!(listvals)
            tmp=pop!(listnodes)
            freeObj(model2, obj)
        end
        if theval==1.0
            lockObj(model2, obj, 0.0)
            push!(listobjs,obj)
            push!(listvals,0.0)
        else
            println("\nFINISHED")
            stop=true
        end
    else
        #the root node was sondable
        println("\nFINISHED")
        stop=true
    end
    return listobjs, listvals, listnodes, stop 
end

ExplorerAutreNoeud_depthfirst! (generic function with 1 method)

###  Création de la relaxation linéaire (= modèle associé au noeud 0): 

In [21]:
mutable struct objDecision
    value::Float64
    price::Float64
    weight::Float64
    ratio::Float64
    index::Int64
    lock::Bool                  # Lock, the value is choosen, !Lock, the value is not choosen yet

end

mutable struct modelBase
    bound1_2::String             # Choice of the bound
    capacity::Int64
    variables::Array{objDecision, 1}
    bound::Float64
    nbFree::Int64                 # Number of values not locked
    infeasible::Bool
    
end

# Creation du model à partir du choix de la borne, du prix et du poids et des objets et de la capacité du sac
function CreationModele_LP(boundChoice, price,weight, capacity)
    n = length(price)
    variables = Array{objDecision, 1}(undef, n)
    for i = 1:n
        variables[i] = objDecision(0.0, price[i], weight[i], price[i]/weight[i],  i, false)
    end

    return model2 = modelBase(boundChoice, capacity, variables, -1, n, false)
end

# Calcul du prix actuel contenu dans le sac
function actualPrice(model)
    return sum([obj.value * obj.price for obj in model.variables])
end

# Calcul du poids actuel contenu dans le sac
function actualWeight(model)
    sum([obj.value*obj.weight for obj in model.variables])
end

# Verrouillage d'un objet lorsque l'on choisi d'utiliser celui-ci
function lockObj(model, obj, value)
    model.nbFree -= 1
    obj.lock = true
end

# Libération d'un objet lorsque l'on retourne en arrière dans l'arbre
function freeObj(model, obj)
    obj.lock = false
    model.nbFree += 1
end

function verifTR(model)
    if (model.nbFree == 1)
        i, index = 1, 0
        while((i<= length(model.variables)) && (index ==0))
            obj = model.variables[i]
            if !obj.lock
                index=i
                residualCapacity = model.capacity - actualWeight(model)
                obj.value = obj.weight <=  residualCapacity  ? 1.0 : 0.0
            end
            i += 1
        end
    end
    return (model.nbFree == 1)
end



verifTR (generic function with 1 method)

In [17]:
function resolution(model)
    residualCapacity = model.capacity - actualWeight(model)
    infeasible = residualCapacity < 0
    bound = -1

    if !infeasible
        #Calcul du prix actuel se trouvant dans le sac
        bound = actualPrice(model)
        bestObj = model.variables[1] # Objet ayant le meilleur ratio
        for obj in model.variables
            if obj.ratio > bestObj.ratio && !obj.lock
                bestObj = obj
            end
        end
        bound += residualCapacity * bestObj.ratio
        println(" Voici la bound potentiel AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA")
        println(bound)
    end
    model.infeasible = infeasible
    model.bound = bound
end


resolution (generic function with 1 method)

### Boucle principale : résoudre une relaxation, appliquer les tests de sondabilité, identifier le prochain noeud, répéter.

In [7]:

function SolveKnapInstance(filename)

    price, weight, capacity = readKnaptxtInstance(filename)

    model2 = CreationModele_LP("simpl", price, weight, capacity)

    #create the structure to memorize the search tree for visualization at the end
    trParentnodes=Int64[] #will store orig node of arc in search tree
    trChildnodes=Int64[] #will store destination node of arc in search tree
    trNamenodes=[] #will store names of nodes in search tree

    #intermediate structure to navigate in the search tree
    listobjs=[]
    listvals=[]
    listnodes=[]

    BestProfit=-1
    Bestsol=[]

    current_node_number=0
    stop = false

    while(!stop)

        println("\nNode number ", current_node_number, ": \n---------------\n")

        #Update the graphical tree
        push!(trNamenodes,current_node_number+1) 
        if(length(trNamenodes)>=2)
            push!(trParentnodes,listnodes[end]+1) # +1 because the 1st node is "node 0"
            push!(trChildnodes, current_node_number+1) # +1 because the 1st node is "node 0"
        end
        push!(listnodes, current_node_number)

        
        #create LP of current node
        resolution(model2)
        
    
        
        #print("Solve the LP model of the current node to compute its bound: start ... ")
        #status = optimize!(model2)
        #println("... end"); 

        print(": Solution LP")
        if(model2.infeasible)#(has_values(model2))
            print(" : NOT AVAILABLE (probably infeasible or ressources limit reached)")
        else
            print(" ", actualPrice(model2))
            [print("\t", v.price,"=",v.value) for v in model2.variables] 
        end
        println(" "); 


        println("\nPrevious Solution memorized ", Bestsol, " with bestprofit ", BestProfit, "\n")

        TA, TO, TR, Bestsol, BestProfit = TestsSondabilite_LP(model2, BestProfit, Bestsol)

        is_node_sondable = TA || TO || TR

        #Reset_LP!(model2, x, listobjs)

        if(!is_node_sondable)
            listobjs, listvals = SeparerNoeud_lexicographic_depthfirst!(model2, listobjs, listvals, length(price))
        else
            listobjs, listvals, listnodes, stop = ExplorerAutreNoeud_depthfirst!(model2, listobjs, listvals, listnodes)
        end

        current_node_number = current_node_number + 1
    end

    println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", Bestsol)

    [print("\t", v.index ,"=",v.value) for v in model2.variables]
    return BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes

end


SolveKnapInstance (generic function with 1 method)

### Affichage du résultat final

In [22]:
BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes = SolveKnapInstance("instancesETU/KNAPnewformat/test.opb.txt")
println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", Bestsol)
graphplot(trParentnodes, trChildnodes, names=trNamenodes, method=:tree)


Node number 0: 
---------------

 Voici la bound potentiel AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
100.0
: Solution LP 0.0	42.0=0.0	40.0=0.0	12.0=0.0	25.0=0.0 

Previous Solution memorized Any[] with bestprofit -1

non sondable

branch on object objDecision(1.0, 40.0, 4.0, 10.0, 2, false)


Node number 1: 
---------------

 Voici la bound potentiel AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
100.0
: Solution LP 40.0	42.0=0.0	40.0=1.0	12.0=0.0	25.0=0.0 

Previous Solution memorized Any[] with bestprofit -1

non sondable

branch on object objDecision(1.0, 40.0, 4.0, 10.0, 2, false)


Node number 2: 
---------------

 Voici la bound potentiel AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
100.0
: Solution LP 40.0	42.0=0.0	40.0=1.0	12.0=0.0	25.0=0.0 

Previous Solution memorized Any[] with bestprofit -1

non sondable

branch on object objDecision(1.0, 40.0, 4.0, 10.0, 2, false)


Node



branch on object objDecision(1.0, 40.0, 4.0, 10.0, 2, false)


Node number 31: 
---------------

 Voici la bound potentiel AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
100.0
: Solution LP 40.0	42.0=0.0	40.0=1.0	12.0=0.0	25.0=0.0 

Previous Solution memorized Any[] with bestprofit -1

non sondable

branch on object objDecision(1.0, 40.0, 4.0, 10.0, 2, false)


Node number 32: 
---------------

 Voici la bound potentiel AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
100.0
: Solution LP 40.0	42.0=0.0	40.0=1.0	12.0=0.0	25.0=0.0 

Previous Solution memorized Any[] with bestprofit -1

non sondable

branch on object objDecision(1.0, 40.0, 4.0, 10.0, 2, false)


Node number 33: 
---------------

 Voici la bound potentiel AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
100.0
: Solution LP 40.0	42.0=0.0	40.0=1.0	12.0=0.0	25.0=0.0 

Previous Solution memorized Any[] with bestprofit -1

non sondable


0.0	25.0=0.0 

Previous Solution memorized Any[] with bestprofit -1

non sondable

branch on object objDecision(1.0, 40.0, 4.0, 10.0, 2, false)


Node number 62: 
---------------

 Voici la bound potentiel AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
100.0
: Solution LP 40.0	42.0=0.0	40.0=1.0	12.0=0.0	25.0=0.0 

Previous Solution memorized Any[] with bestprofit -1

non sondable

branch on object objDecision(1.0, 40.0, 4.0, 10.0, 2, false)


Node number 63: 
---------------

 Voici la bound potentiel AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
100.0
: Solution LP 40.0	42.0=0.0	40.0=1.0	12.0=0.0	25.0=0.0 

Previous Solution memorized Any[] with bestprofit 

-1

non sondable

branch on object objDecision(1.0, 40.0, 4.0, 10.0, 2, false)


Node number 64: 
---------------

 Voici la bound potentiel AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
100.0
: Solution LP 40.0	42.0=0.0	40.0=1.0	12.0=0.0	25.0=0.0 

Previous Solution memorized Any[] with bestprofit -1

non sondable

branch on object objDecision(1.0, 40.0, 4.0, 10.0, 2, false)


Node number 65: 
---------------

 Voici la bound potentiel AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
100.0
: Solution LP 40.0	42.0=0.0	40.0=1.0	12.0=0.0	25.0=0.0 

Previous Solution memorized Any[] with bestprofit -1

non sondable

branch on object objDecision(1.0, 40.0, 4.0, 10.0, 2, false)


Node number 66: 
---------------

 Voici la bound potentiel AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
100.0
: Solution LP 40.0	42.0=0.0	40.0=1.0	12.0=0.0	25.0=0.0 

Previous Solution memorized Any[] with bestprofit -

0.0	40.0=1.0	12.0=0.0	25.0=0.0 

Previous Solution memorized Any[] with bestprofit -1

non sondable

branch on object objDecision(1.0, 40.0, 4.0, 10.0, 2, false)


Node number 95: 
---------------

 Voici la bound potentiel AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
100.0
: Solution LP 40.0	42.0=0.0	40.0=1.0	12.0=0.0	25.0=0.0 

Previous Solution memorized Any[] with bestprofit -1

non sondable

branch on object objDecision(1.0, 40.0, 4.0, 10.0, 2, false)


Node number 96: 
---------------

 Voici la bound potentiel AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
100.0
: Solution LP 40.0	42.0=0.0	40.0=1.0	12.0=0.0	25.0=0.0 

Previous Solution memorized Any[] with bestprofit -1

non sondable

branch on object objDecision(1.0, 40.0, 4.0, 10.0, 2, false)


Node number 97: 
---------------

 Voici la bound potentiel AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
100.0
: Solution LP 40.0	42.0=0

...

2, false)


Node number 1476: 
---------------

 Voici la bound potentiel AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
100.0
: Solution LP 40.0	42.0=0.0	40.0=1.0	12.0=0.0	25.0=0.0 

Previous Solution memorized Any[] with bestprofit -1

non sondable

branch on object objDecision(1.0, 40.0, 4.0, 10.0, 2, false)


Node number 1477: 
---------------

 Voici la bound potentiel AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
100.0
: Solution LP 40.0	42.0=0.0	40.0=1.0	12.0=0.0	25.0=0.0 

Previous Solution memorized Any[] with bestprofit -1

non sondable

branch on object objDecision(1.0, 40.0, 4.0, 10.0, 2, false)


Node number 1478: 
---------------

 Voici la bound potentiel AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
100.0
: Solution LP 40.0	42.0=0.0	40.0=1.0	12.0=0.0	25.0=0.0 

Previous Solution memorized Any[] with bestprofit -1

non sondable

branch on object objDecision(1.0, 40.0, 4.0, 1

1.0	12.0=0.0	25.0=0.0 

Previous Solution memorized Any[] with bestprofit -1

non sondable

branch on object objDecision(1.0, 40.0, 4.0, 10.0, 2, false)


Node number 1507: 
---------------

 Voici la bound potentiel AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
100.0
: Solution LP 40.0	42.0=0.0	40.0=1.0	12.0=0.0	25.0=0.0 

Previous Solution memorized Any[] with bestprofit -1

non sondable

branch on object objDecision(1.0, 40.0, 4.0, 10.0, 2, false)


Node number 1508: 
---------------

 Voici la bound potentiel AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
100.0
: Solution LP 40.0	42.0=0.0	40.0=1.0	12.0=0.0	25.0=0.0 

Previous Solution memorized Any[] with bestprofit -1

non sondable

branch on object objDecision(1.0, 40.0, 4.0, 10.0, 2, false)


Node number 1509: 
---------------

 Voici la bound potentiel AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
100.0
: Solution LP 40.0	42.0=0.0	

Excessive output truncated after 524298 bytes.

: 
---------------

 Voici la bound potentiel AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
100.0
: Solution LP 40.0	42.0=0.0	40.0=1.0	12.0=0.0	25.0=0.0 

Previous Solution memorized Any[] with bestprofit -1

non sondable

branch on object objDecision(1.0, 40.0, 4.0, 10.0, 2, false)


Node number 1521: 
---------------

 Voici la bound potentiel AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
100.0
: Solution LP 40.0	42.0=0.0	40.0=1.0	12.0=0.0	25.0=0.0 

Previous Solution memorized Any[] with bestprofit -1

non sondable

branch on object objDecision(1.0, 40.0, 4.0, 10.0, 2, false)


Node number 1522: 
---------------

 Voici la bound potentiel AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
100.0
: Solution LP 40.0	42.0=0.0	40.0=1.0	12.0=0.0	25.0=0.0 

Previous Solution memorized Any[] with bestprofit -1

non sondable

branch on object objDecision(1.0, 40.0, 4.0, 10.0, 2, false)


Node number

InterruptException: InterruptException: